<h1 align="center">The Transformer Model</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Aqui estudaremos os maravilhosos tutoriais de [Machine Learning Mastery - PhD.: Jason Brownlee](https://machinelearningmastery.com/). Aqui em específico começaremos com os seguintes:

* [The Transformer Model](https://machinelearningmastery.com/the-transformer-model/)

* [Implementing the Transformer Encoder from Scratch in TensorFlow and Keras](https://machinelearningmastery.com/implementing-the-transformer-encoder-from-scratch-in-tensorflow-and-keras)

* [Implementing the Transformer Decoder from Scratch in TensorFlow and Keras](https://machinelearningmastery.com/implementing-the-transformer-decoder-from-scratch-in-tensorflow-and-keras)

* [Joining the Transformer Encoder and Decoder Plus Masking](https://machinelearningmastery.com/joining-the-transformer-encoder-and-decoder-and-masking)

* [Training the Transformer Model](https://machinelearningmastery.com/training-the-transformer-model/)

Já nos familiarizamos com o conceito de `self-attention` (auto-atenção) conforme implementado pelo mecanismo de atenção Transformer para tradução automática neural. Agora vamos mudar nosso foco para os detalhes da **própria arquitetura do Transformer** para descobrir como a auto-atenção pode ser implementada sem depender do uso de `recorrência` e `convoluções`.

Aqui aprenderemos sobre a arquitetura de rede do modelo `Transformer`. Os pontos específicos são:


* Como a arquitetura Transformer implementa uma estrutura codificador-decodificador sem `recorrência` e `convoluções`

* Como funcionam o codificador e o decodificador Transformer 

* Como a autoatenção do Transformer se compara ao uso de camadas recorrentes e convolucionais 


Antes de abordar a leitura deste script, recomendamos ler, também, os seguintes tutoriais:

* [O conceito de Attention](https://machinelearningmastery.com/what-is-attention/)

* [O mecanismo de Attention](https://machinelearningmastery.com/the-attention-mechanism-from-scratch/)

* [O mecanismo de Attention do Transformer](https://machinelearningmastery.com/the-transformer-attention-mechanism)

# A arquitetura do Transformer

A arquitetura do Transformer segue uma estrutura de codificador-decodificador, mas não depende de recorrência e convoluções para gerar uma saída. 

![](https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1-727x1024.png)

Resumindo, a tarefa do codificador, na metade esquerda da arquitetura do Transformer, é mapear uma sequência de entrada para uma sequência de representações contínuas, que é então alimentada em um decodificador. 

O decodificador, na metade direita da arquitetura, recebe a saída do codificador junto com a saída do decodificador no intervalo de tempo anterior para gerar uma sequência de saída.



# O Codificador (The Encoder)

![](https://machinelearningmastery.com/wp-content/uploads/2021/10/transformer_1-727x1024.png)

![](https://i0.wp.com/kikaben.com/wp-content/uploads/2022/04/image-449.png?resize=800%2C724&ssl=1)

O `codificador` (Encoder) consiste em uma pilha de $N=6$ camadas idênticas, onde cada camada é composta por duas subcamadas:

<font color="red">1.</font> A primeira subcamada implementa um mecanismo de auto-atenção (`self-attention`) de múltiplas cabeças. [Você viu](https://machinelearningmastery.com/the-transformer-attention-mechanism) que o mecanismo de várias cabeças implementa $h$ cabeças que recebem uma versão (diferente) projetada linearmente das consultas (queries), chaves e valores, cada uma para produzir $h$ saídas em paralelo que são usadas para gerar um resultado final.

<font color="red">2.</font> A segunda subcamada é uma rede `feed-forward` totalmente conectada (`fully connected`) que consiste em duas transformações lineares com uma função de ativação de `Unidade Linear Retificada` (`ReLU`) entre elas:

$$
FFN(x) = ReLU(W_1 x + b_1)W_2 + b_2
$$

As seis camadas do `codificador Transformer` aplicam as mesmas transformações lineares a todas as palavras na sequência de entrada, mas cada camada emprega um peso diferente $(W_1, W_2)$ e bias $(b_1, b_2)$ parâmetros para fazer isso. 

Além disso, cada uma dessas duas subcamadas possui uma `conexão residual` ao seu redor.

Cada subcamada também é sucedida por uma `camada de normalização`, $layernorm( . )$ que normaliza a soma calculada entre a entrada da subcamada, $x$, e a saída gerada pela própria subcamada, $sublayer(x)$:


$$
layernorm(x + sublayer(x))
$$


<font color="orange">Uma consideração importante a ter em mente é que a arquitetura do Transformer não pode capturar inerentemente nenhuma informação sobre as posições relativas das palavras na sequência, pois não faz uso de `recorrência`</font>. Essas informações devem ser injetadas `introduzindo codificações posicionais` nos Embeddings de entrada. 

Os vetores de codificação (Encoding) posicional são da mesma dimensão que os Embeddings de entrada e são gerados usando funções `seno` e `cosseno` de diferentes frequências. Então, eles são simplesmente somados aos Embeddings de entrada para injetar a informação posicional.

# O Decodificador (The Decoder) 

![](https://machinelearningmastery.com/wp-content/uploads/2021/10/transformer_2-727x1024.png)

![](https://i0.wp.com/kikaben.com/wp-content/uploads/2022/04/image-448.png?resize=800%2C629&ssl=1)

O decodificador (`Decoder`) compartilha várias semelhanças com o codificador (`Encoder`). 

O decodificador também consiste em uma pilha de $N=6$ camadas idênticas, cada uma composta por três subcamadas:

<font color="red">1.</font> A primeira subcamada recebe a saída anterior da pilha do decodificador, aumenta-a com informações posicionais e implementa a auto-atenção de várias cabeças sobre ela. Enquanto o codificador é projetado para atender a todas as palavras na sequência de entrada, independentemente de sua posição na sequência, o decodificador é modificado para atender apenas às palavras anteriores. Portanto, a predição de uma palavra na posição <font color="orange">$i$</font> só pode depender das saídas conhecidas para as palavras que vêm antes dele na sequência.  No mecanismo de atenção multicabeça (<font color="orange">que implementa múltiplas funções de atenção única em paralelo</font>), isso é obtido pela introdução de uma máscara (`mask`) sobre os valores produzidos pela multiplicação escalonada de matrizes $Q$ e $K$. Esse mascaramento (Masking) é implementado suprimindo os valores da matriz que, de outra forma, corresponderiam a conexões ilegais:


 $$
\text{mask}(\mathbf{QK}^T) =


\text{mask}

 \left( \begin{bmatrix}

e_{11} & e_{12} & \dots & e_{1n} \\

e_{21} & e_{22} & \dots & e_{2n} \\

\vdots & \vdots & \ddots & \vdots \\

e_{m1} & e_{m2} & \dots & e_{mn} \\

\end{bmatrix} \right)

=

\begin{bmatrix}

e_{11} & -\infty & \dots & -\infty \\

e_{21} & e_{22} & \dots & -\infty \\

\vdots & \vdots & \ddots & \vdots \\

e_{m1} & e_{m2} & \dots & e_{mn} \\

\end{bmatrix}
$$



![](https://machinelearningmastery.com/wp-content/uploads/2021/09/tour_3-178x300.png)

<font color="red">2.</font> A segunda camada implementa um mecanismo de auto-atenção de múltiplas cabeças semelhante ao implementado na primeira subcamada do codificador (`Encoder`).  No lado do decodificador, esse mecanismo multicabeça recebe as consultas (queries) da subcamada do decodificador (Decoder) anterior e as chaves e valores da saída do codificador. Isso permite que o decodificador atenda a todas as palavras na sequência de entrada.


<font color="red">2.</font>3.</font> A terceira camada implementa uma `rede feed-forward` totalmente conectada, semelhante à implementada na segunda subcamada do codificador.

Além disso, as três subcamadas do lado do decodificador também `possuem conexões residuais` ao seu redor e são sucedidas por uma camada de normalização.

As codificações posicionais também são adicionadas aos Embeddings de entrada do decodificador da mesma maneira explicada anteriormente para o codificador. 

# Resumo: o modelo de Transformer

O modelo Transformer é executado da seguinte forma:

* Cada palavra que forma uma sequência de entrada é transformada em um vetor de Embedding $d_{model}-dimensional$.

* Cada vetor de Embedding que representa uma palavra de entrada é aumentado pela soma (por elementos) a um vetor de codificação posicional do mesmo comprimento $d_{model}$, introduzindo assim informações posicionais na entrada.

* Os vetores de Embeddings aumentados são alimentados no bloco codificador que consiste nas duas subcamadas explicadas acima. Como o codificador atende a todas as palavras na sequência de entrada, independentemente se elas precedem ou sucedem a palavra em consideração, `o codificador Transformer é bidirecional`.

* O decodificador recebe como entrada sua própria palavra de saída prevista no intervalo de tempo, $t-1$.

* A entrada para o decodificador também é aumentada pela codificação posicional da mesma maneira feita no lado do codificador. 

* A entrada do decodificador aumentado é alimentada nas três subcamadas que compreendem o bloco decodificador explicado acima. <font colo="orange">O `mascaramento` é aplicado na primeira subcamada para impedir que o decodificador atente para as palavras seguintes</font>. Na segunda subcamada, o decodificador também recebe a saída do codificador, que agora permite que o decodificador atenda a todas as palavras na sequência de entrada.

* A saída do decodificador finalmente passa por uma camada totalmente conectada (`fully connected`), seguida por uma `camada softmax`, para gerar uma previsão para a próxima palavra da sequência de saída. 

# Comparação com Camadas Recorrentes e Convolucionais

[Vaswani et al. (2017)](https://arxiv.org/abs/1706.03762) explicam que sua motivação para abandonar o uso de recorrência e convoluções foi baseada em vários fatores:

* Verificou-se que as camadas de auto-atenção são mais rápidas do que as camadas `recorrentes` para comprimentos de sequência mais curtos e podem ser restritas para considerar apenas uma vizinhança na sequência de entrada para comprimentos de sequência muito longos.

* O número de operações sequenciais exigidas por uma `camada recorrente` é baseado no comprimento da sequência, enquanto esse número permanece constante para uma camada de auto-atenção.

* Em `redes neurais convolucionais`, a largura do kernel afeta diretamente as dependências de longo prazo que podem ser estabelecidas entre pares de posições de entrada e saída. O rastreamento (`Tracking`) de dependências de longo prazo exigiria o uso de kernels grandes ou pilhas de camadas convolucionais que poderiam aumentar o custo computacional.